In [1]:
pip install nltk

In [2]:
import nltk

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import pandas as pd

In [5]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [13]:
fake['target'] = 0
true['target'] = 1

data = pd.concat([fake, true], axis=0)
data = data.reset_index(drop = True)
data = data.drop(['subject', 'date', 'title'], axis = 1)

In [14]:
print(data.columns)
print(data)

Index(['text', 'target'], dtype='object')
                                                    text  target
0      Donald Trump just couldn t wish all Americans ...       0
1      House Intelligence Committee Chairman Devin Nu...       0
2      On Friday, it was revealed that former Milwauk...       0
3      On Christmas day, Donald Trump announced that ...       0
4      Pope Francis used his annual Christmas Day mes...       0
...                                                  ...     ...
44893  BRUSSELS (Reuters) - NATO allies on Tuesday we...       1
44894  LONDON (Reuters) - LexisNexis, a provider of l...       1
44895  MINSK (Reuters) - In the shadow of disused Sov...       1
44896  MOSCOW (Reuters) - Vatican Secretary of State ...       1
44897  JAKARTA (Reuters) - Indonesia will buy 11 Sukh...       1

[44898 rows x 2 columns]


In [15]:
from nltk.tokenize import word_tokenize
data['text'] = data['text'].apply(word_tokenize)

In [16]:
from nltk.stem.snowball import SnowballStemmer
porter = SnowballStemmer('english', ignore_stopwords = False)

In [17]:
def stem_it(text):
  return [porter.stem(word) for word in text]

In [18]:
data['text'] = data['text'].apply(stem_it)

In [19]:
def stop_it(t):
  dt = [word for word in t if len(word)>2]
  return dt

In [20]:
data['text'] = data['text'].apply(stop_it)
data['text'] = data['text'].apply(' '.join)


In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size = 0.25)


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
my_tfidf = TfidfVectorizer( max_df = 0.7)

tfidf_train = my_tfidf.fit_transform(x_train)
tfidf_test = my_tfidf.transform(x_test)

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [28]:
model_1 = LogisticRegression(max_iter = 900)
model_1.fit(tfidf_train, y_train)
pred_1 = model_1.predict(tfidf_test)
cr1 = accuracy_score(y_test, pred_1)
print(cr1*100)

98.73496659242763


In [29]:
from sklearn.linear_model import PassiveAggressiveClassifier

model = PassiveAggressiveClassifier(max_iter = 50)
model.fit(tfidf_train, y_train)

PassiveAggressiveClassifier(max_iter=50)

In [30]:
y_pred = model.predict(tfidf_test)
accscore = accuracy_score(y_test, y_pred)
print(f"Accuracy = {accscore*100}")

Accuracy = 99.54565701559021
